In [170]:
# Standard Libraries
import os
import random
import pydicom
from PIL import Image
import yaml

# Data Manipulation Libraries
import pandas as pd
import numpy as np


# Machine Learning Libraries
import torch
from sklearn.model_selection import train_test_split

# YOLO library
from ultralytics import YOLO

**Define Parameters**

In [171]:
# Disease labels
disease_labels = [
    "Aortic enlargement", "Atelectasis", "Calcification", "Cardiomegaly",
    "Consolidation", "ILD", "Infiltration", "Lung Opacity",
    "Nodule/Mass", "Other lesion", "Pleural effusion", "Pleural thickening",
    "Pneumothorax", "Pulmonary fibrosis"
]

# File path
EXTRACTED_PATH = '/cluster/home/bjorneme/projects/Data/vinbigdata-chest-xray-abnormalities-detection-extracted'

# Define parameters
SEED = 42
NUM_WORKERS =32

# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


**Set Seed or Reproducibity**

In [172]:
def seed_everything(seed=SEED):
    """
    Sets the seed to ensure reproducibility.
    """
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Apply the seed
seed_everything()

# **Helper Functions**

In [173]:
def open_dicom_image(path):
    ds = pydicom.dcmread(path)
    img = ds.pixel_array.astype(np.float32)
    img = ((img - img.min()) / (img.max() - img.min()) * 255).astype(np.uint8)
    return Image.fromarray(img)

In [174]:
def convert_annotation_to_yolo(group, img_size_override=None):
    img_path = group.iloc[0]['Path']
    img_width, img_height = open_dicom_image(img_path).size
    
    lines = []
    for _, row in group.iterrows():
        x_min, y_min, x_max, y_max = row[['x_min','y_min','x_max','y_max']]
        x_center = ((x_min + x_max) / 2) / img_width
        y_center = ((y_min + y_max) / 2) / img_height
        box_width = (x_max - x_min) / img_width
        box_height = (y_max - y_min) / img_height
        lines.append(f"{int(row['class_id'])} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}")
    return img_path, lines

In [175]:
def process_split(df, split_name, base_dir):
    for image_id, group in df.groupby('image_id'):
        img_path, yolo_lines = convert_annotation_to_yolo(group)
        img_filename = os.path.basename(img_path)
        ext = os.path.splitext(img_filename)[1].lower()
        img_dir = os.path.join(base_dir, "images", split_name)
        label_dir = os.path.join(base_dir, "labels", split_name)
        os.makedirs(img_dir, exist_ok=True)
        os.makedirs(label_dir, exist_ok=True)
        
        pil_img = open_dicom_image(img_path)
        img_filename = os.path.splitext(img_filename)[0] + ".png"
        pil_img.save(os.path.join(img_dir, img_filename))
        
        label_filename = os.path.splitext(img_filename)[0] + ".txt"
        with open(os.path.join(label_dir, label_filename), "w") as f:
            f.write("\n".join(yolo_lines))

# **Step 1: Load Data**

In [176]:
for split in ['train', 'val']:
    os.makedirs(os.path.join(EXTRACTED_PATH, "images", split), exist_ok=True)
    os.makedirs(os.path.join(EXTRACTED_PATH, "labels", split), exist_ok=True)

print("Processing training set...")
# process_split(train_df, "train", EXTRACTED_PATH)
print("Processing validation set...")
# process_split(val_df, "val", EXTRACTED_PATH)

data_yaml = {
    "train": os.path.join(os.getcwd(), EXTRACTED_PATH, "images", "train"),
    "val": os.path.join(os.getcwd(), EXTRACTED_PATH, "images", "val"),
    "nc": 14,
    "names": disease_labels
}

with open("data.yaml", "w") as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

Processing training set...
Processing validation set...


# **Step 2: Data Preprocessing**

In [177]:
def load_labels(csv_path, image_path):
    """
    Loads and preprocesses the labels from the CSV file.
    Maps each image to its corresponding file path and binary labels for each disease.
    """

    # Read the CSV file containing labels
    labels_df = pd.read_csv(csv_path)

    # Create binary columns for each disease label
    for disease in disease_labels:
        labels_df[disease] = labels_df['class_name'].str.contains(disease).astype(int)
    
    # Create a binary column for 'No Finding'
    labels_df['No finding'] = labels_df['class_name'].apply(lambda x: 1 if 'No finding' in x else 0)

    # Add the full image path
    labels_df['Path'] = labels_df['image_id'].apply(
        lambda x: os.path.join(image_path, 'train', f"{x}.dicom")
    )

    return labels_df

# Path to the labels CSV file
labels_csv_path = os.path.join(EXTRACTED_PATH, 'train.csv')

# Load and preprocess the labels
labels_df = load_labels(labels_csv_path, EXTRACTED_PATH)

**Filter Images with Finding**

In [178]:
labels_df = labels_df[labels_df['class_id'] != 14]

**Split Dataset**

In [179]:
# Split based on image id
unique_ids = labels_df['image_id'].unique()

# Split patients into training and validation
train_ids, val_ids = train_test_split(
    unique_ids, test_size=0.2, random_state=SEED
)

# Create dataframes for training and validation sets
train_df = labels_df[labels_df['image_id'].isin(train_ids)]
val_df = labels_df[labels_df['image_id'].isin(train_ids)]

# Verify Split Sizes
print(f"Train size: {len(train_ids)}")
print(f"Val size: {len(val_ids)}")

Train size: 3515
Val size: 879


# **Step 3: Build the Model**

In [180]:
yolo_model = YOLO("yolo11n.pt")
yolo_model.to(device)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_

In [181]:
# --- Import BYOL components from lightly ---
from lightly.loss import NegativeCosineSimilarity
from lightly.models.modules import BYOLProjectionHead, BYOLPredictionHead
from lightly.models.utils import deactivate_requires_grad, update_momentum
from lightly.transforms.byol_transform import (
    BYOLTransform,
    BYOLView1Transform,
    BYOLView2Transform,
)
from lightly.utils.scheduler import cosine_schedule

In [182]:
from torch import nn
import copy

# --- Define the BYOL module ---
class BYOL(nn.Module):
    def __init__(self, backbone, in_features):
        super().__init__()
        self.backbone = backbone
        # Use the computed in_features for the projection head.
        self.projection_head = BYOLProjectionHead(in_features, 1024, 256)
        self.prediction_head = BYOLPredictionHead(256, 1024, 256)

        self.backbone_momentum = copy.deepcopy(self.backbone)
        self.projection_head_momentum = copy.deepcopy(self.projection_head)

        deactivate_requires_grad(self.backbone_momentum)
        deactivate_requires_grad(self.projection_head_momentum)

    def forward(self, x):
        y = self.backbone(x).flatten(start_dim=1)
        z = self.projection_head(y)
        p = self.prediction_head(z)
        return p

    def forward_momentum(self, x):
        y = self.backbone_momentum(x).flatten(start_dim=1)
        z = self.projection_head_momentum(y)
        return z.detach()


# Extract YOLO backbone (adjust layer indices if needed)
backbone = nn.Sequential(*list(yolo_model.model.model.children())[:8])

# Determine the backbone output dimension using a dummy input:
dummy_input = torch.randn(1, 3, 224, 224).to(device)
with torch.no_grad():
    dummy_features = backbone(dummy_input).flatten(start_dim=1)
backbone_out_dim = dummy_features.shape[-1]
print("Backbone output dimension:", backbone_out_dim)

# Create BYOL model with the corrected in_features:
byol_model = BYOL(backbone, in_features=backbone_out_dim)
byol_model.to(device)

Backbone output dimension: 12544


BYOL(
  (backbone): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3k2(
      (cv1): Conv(
        (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (m):

In [183]:
transform = BYOLTransform(
    view_1_transform=BYOLView1Transform(input_size=224, gaussian_blur=0.0),
    view_2_transform=BYOLView2Transform(input_size=224, gaussian_blur=0.0),
)

In [184]:
import torchvision

path = os.path.join(EXTRACTED_PATH, "images")

dataset = torchvision.datasets.ImageFolder(path, transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True, drop_last=True, num_workers=8)

# --- BYOL pretraining loop ---
criterion = NegativeCosineSimilarity()
optimizer = torch.optim.SGD(byol_model.parameters(), lr=0.06)
epochs = 10
print("Starting BYOL pretraining")
for epoch in range(epochs):
    total_loss = 0
    momentum_val = cosine_schedule(epoch, epochs, 0.996, 1)
    for (x0, x1), _ in dataloader:  # ignore labels from ImageFolder
        # Update momentum versions of the backbone and projection head
        update_momentum(byol_model.backbone, byol_model.backbone_momentum, m=momentum_val)
        update_momentum(byol_model.projection_head, byol_model.projection_head_momentum, m=momentum_val)
        x0, x1 = x0.to(device), x1.to(device)
        p0, p1 = byol_model(x0), byol_model(x1)
        z0, z1 = byol_model.forward_momentum(x0), byol_model.forward_momentum(x1)
        loss = 0.5 * (criterion(p0, z1) + criterion(p1, z0))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        break
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch:02d} Loss: {avg_loss:.5f}")

Starting BYOL pretraining


Epoch 00 Loss: 0.00009
Epoch 01 Loss: -0.00058
Epoch 02 Loss: -0.00142
Epoch 03 Loss: -0.00195
Epoch 04 Loss: -0.00241
Epoch 05 Loss: -0.00268
Epoch 06 Loss: -0.00304
Epoch 07 Loss: -0.00333
Epoch 08 Loss: -0.00347
Epoch 09 Loss: -0.00349


In [185]:
pretrained_weights = byol_model.backbone.state_dict()
for idx, layer in enumerate(list(yolo_model.model.model.children())[:8]):
    layer.load_state_dict(pretrained_weights, strict=False)

# **Step 4: Train the Model**

In [186]:
# Train with image size 224 (instead of 640)
results = model.train(data="data.yaml", epochs=10, imgsz=224)

New https://pypi.org/project/ultralytics/8.3.92 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_

Overriding model.yaml nc=80 with nc=14

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytic

train: Scanning /cluster/home/bjorneme/projects/Data/vinbigdata-chest-xray-abnormalities-detection-extracted/labels/train.cache... 3515 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3515/3515 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


/cluster/home/bjorneme/.conda/envs/master_thesis/lib/python3.11/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.21). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
val: Scanning /cluster/home/bjorneme/projects/Data/vinbigdata-chest-xray-abnormalities-detection-extracted/labels/val.cache... 879 images, 0 backgrounds, 0 corrupt: 100%|██████████| 879/879 [00:00<?, ?it/s]


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      10/10      1.37G      1.617      1.558      1.196         71        224: 100%|██████████| 220/220 [00:20<00:00, 10.52it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 28/28 [00:02<00:00,  9.93it/s]


                   all        879       7139      0.313      0.212      0.183     0.0921

10 epochs completed in 0.092 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 5.4MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 5.4MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.80 🚀 Python-3.11.8 torch-2.4.1+cu121 CUDA:0 (NVIDIA A100-SXM4-80GB, 81158MiB)
YOLO11n summary (fused): 100 layers, 2,584,882 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 28/28 [00:03<00:00,  7.35it/s]


                   all        879       7139      0.446      0.204      0.189     0.0935
    Aortic enlargement        610       1446      0.639      0.557      0.615      0.353
           Atelectasis         38         51          0          0      0.131     0.0371
         Calcification         89        189          1          0     0.0138    0.00619
          Cardiomegaly        455       1065      0.579      0.678      0.691       0.44
         Consolidation         61         85      0.281      0.165      0.143     0.0564
                   ILD         74        191      0.175      0.325      0.135     0.0565
          Infiltration        119        240      0.292      0.192      0.179     0.0703
          Lung Opacity        254        465      0.226      0.181      0.128     0.0457
           Nodule/Mass        157        418      0.301     0.0239     0.0262     0.0117
          Other lesion        238        464          1          0     0.0166    0.00554
      Pleural effusio